In [30]:
## REFERENCE: https://www.geeksforgeeks.org/how-to-automate-the-storage-using-dropbox-api-in-python/
import pathlib
import pandas as pd
import dropbox
from dropbox.exceptions import AuthError

In [39]:
TOKEN = '*******'

In [40]:
# Establish connection
def connect_to_dropbox():
    
    try:
        dbx = dropbox.Dropbox(TOKEN)
        print('Connected to Dropbox successfully')
      
    except Exception as e:
        print(str(e))
      
    return dbx
  
dbx = connect_to_dropbox()

Connected to Dropbox successfully


In [107]:
# explicit function to list files
def list_files_in_folder():
    
    # here dbx is an object which is obtained
    # by connecting to dropbox via token
    dbx = connect_to_dropbox()
      
    try:
        filenames = []
        folder_path = "/Audio Submit Form"
  
        # dbx object contains all functions that 
        # are required to perform actions with dropbox
        files = dbx.files_list_folder(folder_path).entries
          
        for file in files:
              
            # listing
            filenames.append(file.name)
        return filenames
              
    except Exception as e:
        print(str(e))

In [112]:
filenames = list_files_in_folder()

Connected to Dropbox successfully
------------Listing Files in Folder------------ 


In [125]:
wav_files = [i for i in filenames if i.endswith('.wav')]
file_num = [int(i.split(" ")[0]) for i in filenames if i.endswith('.wav')]
curr_file = max(file_num)
user_file = [i for i in wav_files if str(curr_file) in i]


In [131]:
user_file[0]

'5390346712512687858 - brown_headed_honeyeater_test1.wav'

In [79]:
import librosa
import numpy as np

def features_extractor(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [132]:
## REFERENCE: https://villoro.com/post/dropbox_python
import io

# explicit function to list files
def read_extract_from_file():
    
    # here dbx is an object which is obtained
    # by connecting to dropbox via token
    dbx = connect_to_dropbox()
      
    try:
        _, res = dbx.files_download("/Audio Submit Form/"+ user_file[0])

        with io.BytesIO(res.content) as stream:
            data=features_extractor(stream)
        return data
            
            
              
    except Exception as e:
        print(str(e))

In [133]:
read_extract_from_file()

Connected to Dropbox successfully


array([-4.9430350e+02,  8.6478668e+01,  3.1200054e+01,  7.0453606e+01,
        9.1567373e+00,  1.6103380e+01,  3.2227825e+01, -4.5723195e+00,
        2.0926613e+01,  1.6201588e+01, -3.6959353e-01,  2.1262352e+01,
        4.0595345e+00,  2.1671052e+00,  1.1584791e+01, -2.1274834e+00,
        4.1524744e+00,  4.4186831e+00, -1.0992719e+00,  5.2339435e+00,
        5.4499042e-01,  9.7623903e-01,  2.2743044e+00, -1.2211622e+00,
        2.4075916e+00, -6.1172299e-02,  1.4788494e-01,  2.5590312e+00,
       -5.7883346e-01,  1.0180750e+00,  4.5317072e-01, -7.3931605e-01,
        1.0377558e+00, -4.5290187e-01,  8.4746951e-01,  4.6483248e-01,
       -5.6448150e-01,  8.7020648e-01, -1.0489001e+00,  1.8393365e-01],
      dtype=float32)